# Modell-Trainer

In [1]:
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import subprocess

class LinuxBefehlTrainer:
    def __init__(self, datensatz_pfad):
        """Initialisiert die Klasse mit dem Pfad zum Datensatz."""
        self.datensatz_pfad = datensatz_pfad
        self.befehle = []
        self.beschreibungen = []
        self.tokenizer = Tokenizer()
        self.max_sequenz_laenge = 0
        self.wort_index = {}
        self.modell = None

    def lade_datensatz(self):
        """Lädt den Datensatz aus der JSON-Datei."""
        with open(self.datensatz_pfad, 'r') as f:
            daten = json.load(f)
        self.befehle = [eintrag['command'] for eintrag in daten]
        self.beschreibungen = [eintrag['description'] for eintrag in daten]

    def verarbeite_daten(self):
        """Verarbeitet die Daten, um sie für das Modell vorzubereiten."""
        self.tokenizer.fit_on_texts(self.beschreibungen)
        sequenzen = self.tokenizer.texts_to_sequences(self.beschreibungen)
        self.max_sequenz_laenge = max(len(seq) for seq in sequenzen)
        self.wort_index = self.tokenizer.word_index
        daten = pad_sequences(sequenzen, maxlen=self.max_sequenz_laenge)
        labels = np.array(range(len(self.befehle)))
        return train_test_split(daten, labels, test_size=0.2, random_state=42)

    def baue_modell(self):
        """Erstellt und kompiliert das Modell für die Vorhersage."""
        self.modell = tf.keras.Sequential([
            tf.keras.layers.Embedding(len(self.wort_index) + 1, 128, input_length=self.max_sequenz_laenge),
            tf.keras.layers.GlobalAveragePooling1D(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(len(self.befehle), activation='softmax')
        ])
        self.modell.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    def trainiere_modell(self, X_train, y_train, X_test, y_test, epochen=10):
        """Trainiert das Modell mit den gegebenen Daten und speichert es."""
        history = self.modell.fit(X_train, y_train, epochs=epochen, validation_data=(X_test, y_test))
        self.modell.save('linux_befehl_modell_de.h5')
        print("Modell erfolgreich trainiert und gespeichert.")



# Befehl-Generator

In [2]:
class LinuxBefehlVorhersager:
    def __init__(self, modell_pfad, datensatz_pfad):
        """Initialisiert die Vorhersageklasse mit dem Modell und Datensatz."""
        self.modell = tf.keras.models.load_model(modell_pfad)
        with open(datensatz_pfad, 'r') as f:
            daten = json.load(f)
        self.befehle = [eintrag['command'] for eintrag in daten]
        self.beschreibungen = [eintrag['description'] for eintrag in daten]
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.beschreibungen)
        self.max_sequenz_laenge = max(len(self.tokenizer.texts_to_sequences([desc])[0]) for desc in self.beschreibungen)

    def sage_befehl_vor(self, beschreibung):
        """Sagt den Linux-Befehl vorher basierend auf der Beschreibung."""
        sequenz = self.tokenizer.texts_to_sequences([beschreibung])
        gepolsterte_sequenz = pad_sequences(sequenz, maxlen=self.max_sequenz_laenge)
        vorhersage = self.modell.predict(gepolsterte_sequenz)
        befehl_index = np.argmax(vorhersage)
        return self.befehle[befehl_index]

# Befehl-Ausführer

In [ ]:
class LinuxBefehlAusfuehrer:
    @staticmethod
    def fuehre_befehl_aus(befehl):
        """Führt den vorhergesagten Befehl aus und gibt die Ausgabe zurück."""
        try:
            result = subprocess.run(befehl, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("Ausgabe:", result.stdout)
            if result.stderr:
                print("Fehler:", result.stderr)
        except subprocess.CalledProcessError as e:
            print("Fehler beim Ausführen des Befehls:", e)

# Training
trainer = LinuxBefehlTrainer('linux_befehle_datensatz_de.json')
trainer.lade_datensatz()
X_train, X_test, y_train, y_test = trainer.verarbeite_daten()
trainer.baue_modell()
trainer.trainiere_modell(X_train, y_train, X_test, y_test, epochen=10)


In [ ]:
# Vorhersage und Ausführung
vorhersager = LinuxBefehlVorhersager('linux_befehl_modell_de.h5', 'linux_befehle_datensatz_de.json')
ausfuehrer = LinuxBefehlAusfuehrer()

while True:
    benutzer_input = input("Geben Sie eine Befehlsbeschreibung ein (oder 'exit' zum Beenden): ")
    if benutzer_input.lower() in ["exit", "quit"]:
        break
    vorhergesagter_befehl = vorhersager.sage_befehl_vor(benutzer_input)
    print(f"Vorhergesagter Befehl: {vorhergesagter_befehl}")
    ausfuehrer.fuehre_befehl_aus(vorhergesagter_befehl)